In [7]:
# instances for directories
TRAIN_new = 'BTD/TRAIN_new/'
VAL_new = 'BTD/VAL_new/'

In [2]:
# Performing Data Augmentation
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import preprocess_input
IMG_SIZE = (224,224)
RANDOM_SEED = 123

train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    TRAIN_new,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary',
    seed=RANDOM_SEED)

validation_generator = test_datagen.flow_from_directory(
    VAL_new,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=16,
    class_mode='binary',
    seed=RANDOM_SEED)

Using TensorFlow backend.


Found 192 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [3]:
# load base model (Using VGG19 model here)
from keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model

vgg19_weight_path = 'vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = VGG19(weights=vgg19_weight_path, include_top=False, input_shape=IMG_SIZE + (3,))

In [5]:
# Buidling model
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras import layers
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.layers[0].trainable = False

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

checkpath = 'weights.{epoch:02d}-{val_loss:.2f}-{val_accuracy:.2f}.hdf5'
cp = ModelCheckpoint(filepath=checkpath,
    monitor="val_accuracy",
    save_weights_only=True,
    save_best_only=True,
    mode='auto')


# Display the model's architecture
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 25088)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 25089     
Total params: 20,049,473
Trainable params: 25,089
Non-trainable params: 20,024,384
_________________________________________________________________


In [6]:
# Training the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=32,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=24,
    callbacks=[cp])

Epoch 1/30
32/32 [==============================] - 312s 10s/step - loss: 3.1056 - accuracy: 0.7129 - val_loss: 8.2735 - val_accuracy: 0.7400
Epoch 2/30
32/32 [==============================] - 235s 7s/step - loss: 1.6611 - accuracy: 0.8613 - val_loss: 1.6355e-06 - val_accuracy: 0.8200
Epoch 3/30
32/32 [==============================] - 236s 7s/step - loss: 1.3237 - accuracy: 0.8760 - val_loss: 6.3245e-23 - val_accuracy: 0.7400
Epoch 4/30
32/32 [==============================] - 234s 7s/step - loss: 1.0943 - accuracy: 0.9043 - val_loss: 3.3171e-16 - val_accuracy: 0.8400
Epoch 5/30
32/32 [==============================] - 234s 7s/step - loss: 0.8633 - accuracy: 0.9023 - val_loss: 1.1814e-07 - val_accuracy: 0.9000
Epoch 6/30
32/32 [==============================] - 235s 7s/step - loss: 0.7388 - accuracy: 0.9238 - val_loss: 2.1962e-14 - val_accuracy: 0.8600
Epoch 7/30
32/32 [==============================] - 235s 7s/step - loss: 0.7374 - accuracy: 0.9287 - val_loss: 1.3047 - val_accuracy:

In [7]:
# Saving the Model
model.save('BTD_VGG19_version06')